##### Copyright 2019 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Bucle de entrenamiento personalizado con Keras y MultiWorkerMirroredStrategy

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/distribute/multi_worker_with_ctl"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/distribute/multi_worker_with_ctl.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/distribute/multi_worker_with_ctl.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tutorials/distribute/multi_worker_with_ctl.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a></td>
</table>

## Descripción general

Este tutorial muestra cómo realizar un entrenamiento distribuido multitrabajador con un modelo Keras y con [bucles de entrenamiento personalizados](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch) utilizando la API `tf.distribute.Strategy`. El bucle de entrenamiento se distribuye a través de `tf.distribute.MultiWorkerMirroredStrategy`, por lo que un modelo `tf.keras` (diseñado para ejecutarse en [un solo trabajador](custom_training.ipynb)) puede funcionar sin problemas en múltiples trabajadores con cambios mínimos en el código. Los bucles de entrenamiento personalizados aportan flexibilidad y un mayor control sobre el entrenamiento, a la vez que facilitan la depuración del modelo. Aprenda más sobre [cómo escribir un bucle de entrenamiento básico](../../guide/basic_training_loops.ipynb), [cómo escribir un bucle de entrenamiento desde cero](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch) y [entrenamiento personalizado](../customization/custom_training_walkthrough.ipynb).

Si está buscando cómo usar `MultiWorkerMirroredStrategy` con `tf.keras.Model.fit`, consulte en su lugar este [tutorial](multi_worker_with_keras.ipynb).

Para aquellos interesados en una comprensión más profunda de las APIs `tf.distribute.Strategy` está disponible la [Guía de entrenamiento distribuido en TensorFlow](../../guide/distributed_training.ipynb) para una visión general de las estrategias de distribución que TensorFlow soporta.

## Preparación

En primer lugar, algunas importaciones necesarias.

In [ ]:
import json
import os
import sys

Antes de importar TensorFlow, realice algunos cambios en el entorno:

- Desactive todas las GPU. Esto evita errores causados por todos los trabajadores tratando de usar la misma GPU. En una aplicación del mundo real, cada trabajador estaría en una máquina diferente.

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

- Restablezca la variable de entorno `'TF_CONFIG'` (verá más sobre esto más adelante).

In [ ]:
os.environ.pop('TF_CONFIG', None)

- Asegúrese de que el directorio actual está en la ruta de Python. Esto permitirá al bloc de notas importar posteriormente los archivos escritos por `%%writefile`.


In [ ]:
if '.' not in sys.path:
  sys.path.insert(0, '.')

Ahora importe TensorFlow.

In [ ]:
import tensorflow as tf

### Definición del conjunto de datos y del modelo

A continuación, cree un archivo `mnist.py` con un modelo simple y la configuración del conjunto de datos. Este archivo Python será usado por los procesos-trabajadores de este tutorial:

In [ ]:
%%writefile mnist.py

import os
import tensorflow as tf
import numpy as np

def mnist_dataset(batch_size):
  (x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
  # The `x` arrays are in uint8 and have values in the range [0, 255].
  # You need to convert them to float32 with values in the range [0, 1]
  x_train = x_train / np.float32(255)
  y_train = y_train.astype(np.int64)
  train_dataset = tf.data.Dataset.from_tensor_slices(
      (x_train, y_train)).shuffle(60000)
  return train_dataset

def dataset_fn(global_batch_size, input_context):
  batch_size = input_context.get_per_replica_batch_size(global_batch_size)
  dataset = mnist_dataset(batch_size)
  dataset = dataset.shard(input_context.num_input_pipelines,
                          input_context.input_pipeline_id)
  dataset = dataset.batch(batch_size)
  return dataset

def build_cnn_model():
  regularizer = tf.keras.regularizers.L2(1e-5)
  return tf.keras.Sequential([
      tf.keras.Input(shape=(28, 28)),
      tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
      tf.keras.layers.Conv2D(32, 3,
                             activation='relu',
                             kernel_regularizer=regularizer),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128,
                            activation='relu',
                            kernel_regularizer=regularizer),
      tf.keras.layers.Dense(10, kernel_regularizer=regularizer)
  ])

## Configuración multitrabajador

Entremos ahora en el mundo del entrenamiento multitrabajador. En TensorFlow, la variable de entorno `'TF_CONFIG'` es necesaria para el entrenamiento en múltiples máquinas. Cada máquina puede tener un rol diferente. La variable `'TF_CONFIG'` que se usa a continuación es una cadena JSON que especifica la configuración del cluster en cada trabajador que forma parte del cluster. Este es el método predeterminado para especificar un clúster, usando `cluster_resolver.TFConfigClusterResolver`, pero hay otras opciones disponibles en el módulo `distribute.cluster_resolver`. Obtenga más información sobre el ajuste de la variable `'TF_CONFIG'` en la [Guía de entrenamiento de distribución](../../guide/distributed_training.ipynb).

### Describa su clúster

Esta es una configuración de ejemplo:

In [ ]:
tf_config = {
    'cluster': {
        'worker': ['localhost:12345', 'localhost:23456']
    },
    'task': {'type': 'worker', 'index': 0}
}

Tenga en cuenta que `tf_config` es sólo una variable local en Python. Para usarla para la configuración del entrenamiento, serialícela como JSON y colóquela en una variable de entorno `'TF_CONFIG'`. Aquí está el mismo `'TF_CONFIG'` serializado como una cadena JSON:

In [ ]:
json.dumps(tf_config)

Hay dos componentes de la `'TF_CONFIG'`: `'cluster'` y `'task'`.

- `'cluster'` es el mismo para todos los trabajadores y da información sobre el cluster de entrenamiento, que es un dict formado por diferentes tipos de trabajos como `'worker'`. En el entrenamiento multitrabajador con `MultiWorkerMirroredStrategy`, suele haber un `'worker'` que asume un poco más de responsabilidad, como guardar puntos de verificación y escribir archivos de resumen para TensorBoard, además de lo que hace un `'worker'` normal. A dicho trabajador se le llama `'chief'`, y es habitual que el `'worker'` con `'index'` 0 sea el designado como `worker` chief o "jefe".

- `'task'` da información de la tarea actual y es diferente en cada trabajador. Especifica el `'type'` y el `'index'` de ese trabajador.

En este ejemplo, se pone el `'type'` de la tarea como `'worker'` y el `'index'` de la tarea como `0`. Esta máquina es el primer trabajador y será designada como trabajador chief y realizará más trabajo que las demás. Tenga en cuenta que las demás máquinas también deberán tener configurada la variable de entorno `'TF_CONFIG'` y deberá tener el mismo dict `'cluster'`, pero diferente `'type'` de tarea o `'index'` de tarea dependiendo de cuáles sean los roles de esas máquinas.


Para fines ilustrativos, este tutorial muestra cómo se puede establecer un `'TF_CONFIG'` con dos trabajadores en `'localhost'`.  En la práctica, los usuarios crearían múltiples trabajadores en direcciones IP/puertos externos, y configurarían `'TF_CONFIG'` en cada trabajador apropiadamente.

Este ejemplo usa dos trabajadores. El `'TF_CONFIG'` del primer trabajador se muestra arriba. Para el segundo trabajador, configure `tf_config['task']['index']=1`.

### Variables de entorno y subprocesos en blocs de notas

Los subprocesos heredan las variables de entorno de su padre. Así que si configura una variable de entorno en este proceso de bloc de notas Jupyter:

In [ ]:
os.environ['GREETINGS'] = 'Hello TensorFlow!'

podrá acceder a la variable de entorno desde un subproceso:

In [ ]:
%%bash
echo ${GREETINGS}

En la siguiente sección, usará esto para pasar el `'TF_CONFIG'` a los subprocesos de los trabajadores. Realmente usted nunca iniciaría sus trabajos de esta manera, pero es suficiente para los propósitos de este tutorial: demostrar un ejemplo mínimo de multitrabajador.

## MultiWorkerMirroredStrategy

Antes de entrenar el modelo, cree primero una instancia de `tf.distribute.MultiWorkerMirroredStrategy`:

In [ ]:
strategy = tf.distribute.MultiWorkerMirroredStrategy()

Nota: `'TF_CONFIG'` se parsea y los servidores GRPC de TensorFlow se inician en el momento en que usted llama a `tf.distribute.MultiWorkerMirroredStrategy.` Por lo tanto, debe establecer la variable de entorno `'TF_CONFIG'` antes de instanciar una `tf.distribute.Strategy`. Para ganar tiempo en este ejemplo ilustrativo, esto no se demuestra en este tutorial, para que no tenga que iniciar los servidores. Puede encontrar un ejemplo completo en la última sección de este tutorial.

Utilice `tf.distribute.Strategy.scope` para especificar que se debe usar una estrategia al construir su modelo. Esto permitirá que la estrategia regule aspectos como la ubicación de las variables: creará copias de todas las variables de las capas del modelo en cada dispositivo de todos los trabajadores.

In [ ]:
import mnist
with strategy.scope():
  # Model building needs to be within `strategy.scope()`.
  multi_worker_model = mnist.build_cnn_model()

## Fragmentar datos automáticamente entre los trabajadores

En el entrenamiento multitrabajador, *se necesita fragmentar el conjunto de datos* para garantizar la convergencia y la reproducibilidad. Fragmentar significa entregar a cada trabajador un subconjunto de todo el conjunto de datos -ayuda a crear una experiencia similar al entrenamiento en un solo trabajador. En el ejemplo siguiente, se utiliza la política predeterminada de autofragmentación de `tf.distribute`. También puede personalizarla configurando la `tf.data.experimental.AutoShardPolicy` de las `tf.data.experimental.DistributeOptions`. Para obtener más información, consulte la sección *Fragmentación* del [Tutorial de entrada distribuida](input.ipynb).

In [ ]:
per_worker_batch_size = 64
num_workers = len(tf_config['cluster']['worker'])
global_batch_size = per_worker_batch_size * num_workers

with strategy.scope():
  multi_worker_dataset = strategy.distribute_datasets_from_function(
      lambda input_context: mnist.dataset_fn(global_batch_size, input_context))

## Definir un bucle de entrenamiento personalizado y entrenar el modelo

Especifique un optimizador:

In [ ]:
with strategy.scope():
  # The creation of optimizer and train_accuracy needs to be in
  # `strategy.scope()` as well, since they create variables.
  optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)
  train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      name='train_accuracy')

Defina un paso de entrenamiento con `tf.function`:


In [ ]:
@tf.function
def train_step(iterator):
  """Training step function."""

  def step_fn(inputs):
    """Per-Replica step function."""
    x, y = inputs
    with tf.GradientTape() as tape:
      predictions = multi_worker_model(x, training=True)
      per_example_loss = tf.keras.losses.SparseCategoricalCrossentropy(
          from_logits=True,
          reduction=tf.keras.losses.Reduction.NONE)(y, predictions)
      loss = tf.nn.compute_average_loss(per_example_loss)
      model_losses = multi_worker_model.losses
      if model_losses:
        loss += tf.nn.scale_regularization_loss(tf.add_n(model_losses))

    grads = tape.gradient(loss, multi_worker_model.trainable_variables)
    optimizer.apply_gradients(
        zip(grads, multi_worker_model.trainable_variables))
    train_accuracy.update_state(y, predictions)
    return loss

  per_replica_losses = strategy.run(step_fn, args=(next(iterator),))
  return strategy.reduce(
      tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None)

### Guardado y restauración del punto de verificación

Al escribir un bucle de entrenamiento personalizado, deberá gestionar [el guardado de puntos de verificación](../../guide/checkpoint.ipynb) manualmente en lugar de confiar en una retrollamada de Keras. Tenga en cuenta que para `MultiWorkerMirroredStrategy`, guardar un punto de verificación o un modelo completo requiere la participación de todos los trabajadores, porque intentar guardar sólo en el trabajador jefe podría llevar a un punto muerto. Los trabajadores también necesitan escribir en rutas diferentes para evitar sobrescribirse unos a otros. Aquí tiene un ejemplo de cómo configurar los directorios:

In [ ]:
from multiprocessing import util
checkpoint_dir = os.path.join(util.get_temp_dir(), 'ckpt')

def _is_chief(task_type, task_id, cluster_spec):
  return (task_type is None
          or task_type == 'chief'
          or (task_type == 'worker'
              and task_id == 0
              and "chief" not in cluster_spec.as_dict()))

def _get_temp_dir(dirpath, task_id):
  base_dirpath = 'workertemp_' + str(task_id)
  temp_dir = os.path.join(dirpath, base_dirpath)
  tf.io.gfile.makedirs(temp_dir)
  return temp_dir

def write_filepath(filepath, task_type, task_id, cluster_spec):
  dirpath = os.path.dirname(filepath)
  base = os.path.basename(filepath)
  if not _is_chief(task_type, task_id, cluster_spec):
    dirpath = _get_temp_dir(dirpath, task_id)
  return os.path.join(dirpath, base)

Cree un `tf.train.Checkpoint` que realice el seguimiento del modelo, administrado por un `tf.train.CheckpointManager`, de forma que sólo se conserven los últimos puntos de verificación:

In [ ]:
epoch = tf.Variable(
    initial_value=tf.constant(0, dtype=tf.dtypes.int64), name='epoch')
step_in_epoch = tf.Variable(
    initial_value=tf.constant(0, dtype=tf.dtypes.int64),
    name='step_in_epoch')
task_type, task_id = (strategy.cluster_resolver.task_type,
                      strategy.cluster_resolver.task_id)
# Normally, you don't need to manually instantiate a `ClusterSpec`, but in this
# illustrative example you did not set `'TF_CONFIG'` before initializing the
# strategy. Check out the next section for "real-world" usage.
cluster_spec = tf.train.ClusterSpec(tf_config['cluster'])

checkpoint = tf.train.Checkpoint(
    model=multi_worker_model, epoch=epoch, step_in_epoch=step_in_epoch)

write_checkpoint_dir = write_filepath(checkpoint_dir, task_type, task_id,
                                      cluster_spec)
checkpoint_manager = tf.train.CheckpointManager(
    checkpoint, directory=write_checkpoint_dir, max_to_keep=1)

Ahora, cuando necesite restaurar un punto de verificación, podrá encontrar el último punto de verificación guardado usando la práctica función `tf.train.latest_checkpoint` (o llamando a `tf.train.CheckpointManager.restore_or_initialize`).

In [ ]:
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
if latest_checkpoint:
  checkpoint.restore(latest_checkpoint)

Tras restablecer el punto de verificación, podrá continuar con el entrenamiento de su bucle de entrenamiento personalizado.

In [ ]:
num_epochs = 3
num_steps_per_epoch = 70

while epoch.numpy() < num_epochs:
  iterator = iter(multi_worker_dataset)
  total_loss = 0.0
  num_batches = 0

  while step_in_epoch.numpy() < num_steps_per_epoch:
    total_loss += train_step(iterator)
    num_batches += 1
    step_in_epoch.assign_add(1)

  train_loss = total_loss / num_batches
  print('Epoch: %d, accuracy: %f, train_loss: %f.'
                %(epoch.numpy(), train_accuracy.result(), train_loss))

  train_accuracy.reset_states()

  # Once the `CheckpointManager` is set up, you're now ready to save, and remove
  # the checkpoints non-chief workers saved.
  checkpoint_manager.save()
  if not _is_chief(task_type, task_id, cluster_spec):
    tf.io.gfile.rmtree(write_checkpoint_dir)

  epoch.assign_add(1)
  step_in_epoch.assign(0)

## Código completo de un vistazo

Para resumir todos los procedimientos analizados hasta ahora:

1. Cree procesos de trabajo.
2. Pase `'TF_CONFIG'`s a los procesos trabajadores.
3. Deje que cada proceso trabajador ejecute el script de abajo que contiene el código de entrenamiento.

In [ ]:
%%writefile main.py
#@title File: `main.py`
import os
import json
import tensorflow as tf
import mnist
from multiprocessing import util

per_worker_batch_size = 64
tf_config = json.loads(os.environ['TF_CONFIG'])
num_workers = len(tf_config['cluster']['worker'])
global_batch_size = per_worker_batch_size * num_workers

num_epochs = 3
num_steps_per_epoch=70

# Checkpoint saving and restoring
def _is_chief(task_type, task_id, cluster_spec):
  return (task_type is None
          or task_type == 'chief'
          or (task_type == 'worker'
              and task_id == 0
              and 'chief' not in cluster_spec.as_dict()))

def _get_temp_dir(dirpath, task_id):
  base_dirpath = 'workertemp_' + str(task_id)
  temp_dir = os.path.join(dirpath, base_dirpath)
  tf.io.gfile.makedirs(temp_dir)
  return temp_dir

def write_filepath(filepath, task_type, task_id, cluster_spec):
  dirpath = os.path.dirname(filepath)
  base = os.path.basename(filepath)
  if not _is_chief(task_type, task_id, cluster_spec):
    dirpath = _get_temp_dir(dirpath, task_id)
  return os.path.join(dirpath, base)

checkpoint_dir = os.path.join(util.get_temp_dir(), 'ckpt')

# Define Strategy
strategy = tf.distribute.MultiWorkerMirroredStrategy()

with strategy.scope():
  # Model building/compiling need to be within `tf.distribute.Strategy.scope`.
  multi_worker_model = mnist.build_cnn_model()

  multi_worker_dataset = strategy.distribute_datasets_from_function(
      lambda input_context: mnist.dataset_fn(global_batch_size, input_context))
  optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)
  train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      name='train_accuracy')

@tf.function
def train_step(iterator):
  """Training step function."""

  def step_fn(inputs):
    """Per-Replica step function."""
    x, y = inputs
    with tf.GradientTape() as tape:
      predictions = multi_worker_model(x, training=True)
      per_example_loss = tf.keras.losses.SparseCategoricalCrossentropy(
          from_logits=True,
          reduction=tf.keras.losses.Reduction.NONE)(y, predictions)
      loss = tf.nn.compute_average_loss(per_example_loss)
      model_losses = multi_worker_model.losses
      if model_losses:
        loss += tf.nn.scale_regularization_loss(tf.add_n(model_losses))

    grads = tape.gradient(loss, multi_worker_model.trainable_variables)
    optimizer.apply_gradients(
        zip(grads, multi_worker_model.trainable_variables))
    train_accuracy.update_state(y, predictions)

    return loss

  per_replica_losses = strategy.run(step_fn, args=(next(iterator),))
  return strategy.reduce(
      tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None)

epoch = tf.Variable(
    initial_value=tf.constant(0, dtype=tf.dtypes.int64), name='epoch')
step_in_epoch = tf.Variable(
    initial_value=tf.constant(0, dtype=tf.dtypes.int64),
    name='step_in_epoch')

task_type, task_id, cluster_spec = (strategy.cluster_resolver.task_type,
                                    strategy.cluster_resolver.task_id,
                                    strategy.cluster_resolver.cluster_spec())

checkpoint = tf.train.Checkpoint(
    model=multi_worker_model, epoch=epoch, step_in_epoch=step_in_epoch)

write_checkpoint_dir = write_filepath(checkpoint_dir, task_type, task_id,
                                      cluster_spec)
checkpoint_manager = tf.train.CheckpointManager(
    checkpoint, directory=write_checkpoint_dir, max_to_keep=1)

# Restoring the checkpoint
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
if latest_checkpoint:
  checkpoint.restore(latest_checkpoint)

# Resume our CTL training
while epoch.numpy() < num_epochs:
  iterator = iter(multi_worker_dataset)
  total_loss = 0.0
  num_batches = 0

  while step_in_epoch.numpy() < num_steps_per_epoch:
    total_loss += train_step(iterator)
    num_batches += 1
    step_in_epoch.assign_add(1)

  train_loss = total_loss / num_batches
  print('Epoch: %d, accuracy: %f, train_loss: %f.'
                %(epoch.numpy(), train_accuracy.result(), train_loss))

  train_accuracy.reset_states()

  checkpoint_manager.save()
  if not _is_chief(task_type, task_id, cluster_spec):
    tf.io.gfile.rmtree(write_checkpoint_dir)

  epoch.assign_add(1)
  step_in_epoch.assign(0)

El directorio actual contiene ahora ambos archivos Python:

In [ ]:
%%bash
ls *.py

Entonces, serialice el `'TF_CONFIG'` a JSON y añádalo a las variables de entorno:

In [ ]:
os.environ['TF_CONFIG'] = json.dumps(tf_config)

Ahora, puede lanzar un proceso trabajador que ejecutará el `main.py` y usará el `'TF_CONFIG'`:

In [ ]:
# first kill any previous runs
%killbgscripts

In [ ]:
%%bash --bg
python main.py &> job_0.log

Hay que tener en cuenta algunas cosas sobre el comando anterior:

1. Usa el `%%bash` que es ["magia" de bloc de notas](https://ipython.readthedocs.io/en/stable/interactive/magics.html) para ejecutar algunos comandos bash.
2. Usa el indicador `--bg` para ejecutar el proceso `bash` en segundo plano, porque este trabajador no terminará. Espera a todos los trabajadores antes de iniciarse.

El proceso trabajador en segundo plano no imprimirá la salida en este bloc de notas. El `&>` redirige su salida a un archivo, para que pueda inspeccionar lo sucedido.

Espere unos segundos a que se inicie el proceso:

In [ ]:
import time
time.sleep(20)

Ahora, revise la salida al archivo de registro del trabajador hasta el momento:

In [ ]:
%%bash
cat job_0.log

La última línea del archivo de registro debería decir: `Started server with target: grpc://localhost:12345`. El primer trabajador ya está listo y está esperando a que todos los demás trabajadores estén listos para continuar.

Actualice el `tf_config` para que el proceso del segundo trabajador prosiga desde ahí:

In [ ]:
tf_config['task']['index'] = 1
os.environ['TF_CONFIG'] = json.dumps(tf_config)

Ahora lance el segundo trabajador. Esto iniciará el entrenamiento ya que todos los trabajadores están activos (por lo que no es necesario poner este proceso en segundo plano):

In [ ]:
%%bash
python main.py > /dev/null 2>&1

Si vuelve a revisar los registros escritos por el primer trabajador, observará que participó en el entrenamiento de ese modelo:

In [ ]:
%%bash
cat job_0.log

In [ ]:
# Delete the `'TF_CONFIG'`, and kill any background tasks so they don't affect the next section.
os.environ.pop('TF_CONFIG', None)
%killbgscripts

## Entrenamiento multitrabajador a fondo

Este tutorial ha demostrado un flujo de trabajo de bucle de entrenamiento personalizado de la configuración multitrabajador. Encontrará descripciones detalladas de otros temas en el tutorial [Entrenamiento multitrabajador con (`tf.keras.Model.fit`) de Keras](multi_worker_with_keras.ipynb) aplicable a los bucles de entrenamiento personalizados.

## Más información

1. La guía [Entrenamiento distribuido en TensorFlow](../../guide/distributed_training.ipynb) proporciona una visión general de las estrategias de distribución disponibles.
2. [Modelos oficiales](https://github.com/tensorflow/models/tree/master/official), muchos de los cuales pueden configurarse para ejecutar múltiples estrategias de distribución.
3. La [sección Rendimiento](../../guide/function.ipynb) de la guía `tf.function` proporciona información sobre otras estrategias y [herramientas](../../guide/profiler.md) que puede usar para optimizar el rendimiento de sus modelos TensorFlow.
